## Закрытие отчетности

### Импорт библиотек

In [21]:
import pandas as pd
from datetime import datetime
from getpass import getpass, getuser
from pathlib import Path
import sys
from importlib import reload
cur_path = str(Path.cwd().parent) # перевести на переменную, когда буду все размещать в сети
home_path = str(Path.cwd().parent.parent)
sys.path.append(home_path + r'\0_sys')
import base_functions as bf
reload(bf)
import glob
import os
import polars as pl
import duckdb as ddb
import gc
import gzip

In [ ]:
# параметры duckdb, не использовать, оставил для справки
ddb.execute(""" 
           SET memory_limit = '2GB';
           SET temp_directory = '/tmp/duckdb_swap';
           SET max_temp_directory_size = '3GB';
           """).df()
duckdb.execute("SET GLOBAL pandas_analyze_sample = 100_000")

## Выгрузка данных

In [10]:
# папки
map_path = r'P:\*\reports\10_sources\mapping' + r'\1_mapping_main_2025.xlsx'
path_root_x = r'P:\*\reports\10_sources\data\1_x_promo_w_geo' # рабочая папка витрины
path_root_rto = r'P:\*\reports\10_sources\data\3_rto' # рабочая папка витрины
# текствоые отбивки
na_txt = 'Не определен'
other_txt = 'Прочее'

In [2]:
# подключения
connection_str = bf.conn_str('edw')

Пароль: ········


* Витрина токенизации
<div class="alert alert-block alert-danger">
<b>TRINO:</b> Сделать подгонку скрипта под Трино и потестить с новым скриптом выгрузки (коммент от 15.10.2025)
</div>

In [4]:
path_root = path_root_x # рабочая папка витрины
sql_name = '1_loyalty_movement_w_promo_w_geo' # файл с sql запросом

In [5]:
%%time
# выполняем запрос
bf.execute_range_queries(connection_str = connection_str, 
                        dt_start = datetime(2025,8,1).date(), # дата начала забора данных по дате движению
                        dt_end = datetime(2025,10,1).date(), # дата окончания забора по дате движения, не включительно
                        path_sql = path_root + '\\sql', # папка в которой хранятся sql запросы
                        qr_name = sql_name, # наименование файла sql запроса
                        path_to_save = path_root + '\\raw', # папка куда сбрасываем выгруженные файлы
                        params = {}, # доп параметры, помимо дат
                        exec_qr = False) # если True, то сначала выполнится запрос с наименование, который был задан здесь с суф _exec (н-р предварительно будет создана временная таблица)

Обработка запроса с выходным файлом - 1_loyalty_movement_w_promo_w_geo_2025-08-01_20251002_2117.csv
Запрос успешно выполнен и сохранен в файл 1_loyalty_movement_w_promo_w_geo_2025-08-01_20251002_2117.csv
-----------------------------------------------------
Обработка запроса с выходным файлом - 1_loyalty_movement_w_promo_w_geo_2025-09-01_20251002_2117.csv
Запрос успешно выполнен и сохранен в файл 1_loyalty_movement_w_promo_w_geo_2025-09-01_20251002_2117.csv
-----------------------------------------------------
CPU times: total: 656 ms
Wall time: 15min 6s


In [6]:
# распределение файлов по папкам (загруженные > подготовленные > архив)
bf.process_reports(path_root)

2025-10-03 12:05:40,290 - ARCHIVED: 1_loyalty_movement_w_promo_w_geo_2025-08-01_20250902_1852.csv
2025-10-03 12:05:40,318 - UPDATED: 1_loyalty_movement_w_promo_w_geo_2025-08-01_20251002_2117.csv
2025-10-03 12:05:40,346 - ADDED: 1_loyalty_movement_w_promo_w_geo_2025-09-01_20251002_2117.csv


* РТО

In [5]:
# переменные
path_root = path_root_rto # рабочая папка витрины
sql_name = '3_rto_w_loyal' # файл с sql запросом

In [ ]:
%%time
# выполняем запрос
bf.execute_range_queries(connection_str = connection_str, 
                        dt_start = datetime(2025,7,1).date(), 
                        dt_end = datetime(2025,9,1).date(), 
                        path_sql = path_root + '\\sql', 
                        qr_name = sql_name,
                        path_to_save = path_root + '\\raw', 
                        params = {}, 
                        exec_qr = True)

In [ ]:
# распределение файлов по папкам (загруженные > подготовленные > архив)
bf.process_reports(path_root)

## Движение баллов, токенизация

#### Соединяем исходники для обработки

In [ ]:
# папка с файлами csv
csv_path = path_root_x + r'\processed\*.csv'
# столбцы
x_promo_schema = {'dt_month':pl.Date,
       'lty_promo_owner_iss_owner_nm':pl.Utf8,
       'lty_promotion_iss_owner_nm':pl.Utf8,
       'loyalty_operation_batch_id':pl.Int32,
       'lty_point_duration_desc':pl.Utf8,
       'lty_point_mvnt_type_desc':pl.Utf8,
       'lty_partner_owner_nm':pl.Utf8,
       'store_tech_owner_desc':pl.Utf8,
       'lty_partner_nm':pl.Utf8,
       'store_tech_mvnt_nm':pl.Utf8,
       'division_nm':pl.Utf8,
       'lty_promotion_action_chain_nm':pl.Utf8,
       'lty_point_rub_amt':pl.Float32}
# создаем единый дф
df_x_promo_all = bf.read_processed_data(csv_path, x_promo_schema)
df_x_promo_all.collect_schema()

In [7]:
# сохраняем периоды, по которым мы потом будем фильтровать и сохранять наши данные кусками
df_x_promo_all_years = sorted(
   df_x_promo_all.select(pl.col("dt_month").dt.year()).unique().collect().to_series().to_list()
)

#### Справочник по патернам

* прочие справочники

In [8]:
# прочие мэппинги
# меппинг тип движения
map_mvnt_type = bf.pl_open_excel(map_path, 'mvnt_type')
# меппинг паттернов
map_promo_nm = bf.pl_open_excel(map_path, 'promo_nm')
# выгружаем все строки по паттернам из бд
qr_manual = bf.get_sql(r'\\*\FolderRedirection-6\*\Desktop\x5_work\projects_files\10_sources\sql','10_batches')
map_manual = bf.pl_read_sql_from_query(connection_str, qr_manual).with_columns(pl.col('loyalty_operation_batch_id').cast(pl.Int64))

* Проверка справочников по паттернам

In [9]:
# создаем таблицу, где проверяются накладываются паттерны и также забираем комменты по id
map_manual = ddb.sql("""
   SELECT
       distinct
       loyalty_operation_batch_id::int64 as loyalty_operation_batch_id
       , dm.batch_comment
       , mp.lty_promotion_iss_owner_nm
       , mp.promo_nm_group
       , mp.pn_group_budget_tcx
       , mp.pn_group_budget_tc5
       , count(distinct mp.lty_promotion_iss_owner_nm) over (partition by dm.batch_comment) cnt_uniq
   FROM
       map_manual dm
   LEFT JOIN
       map_promo_nm mp
           on lower(dm.batch_comment) like concat('%',lower(mp.lty_promotion_iss_owner_nm),'%')
   ORDER BY
       1
""").pl().lazy()
# проверка
map_manual.collect().filter(pl.col('cnt_uniq')>1)

loyalty_operation_batch_id,batch_comment,lty_promotion_iss_owner_nm,promo_nm_group,pn_group_budget_tcx,pn_group_budget_tc5,cnt_uniq
i64,str,str,str,str,str,i64


In [ ]:
map_manual.collect().write_clipboard(separator=';')

#### Проверка, что все элементы в исходнике присутствуют в меппинге

<div class="alert alert-block alert-danger">
<b>Справочники:</b> Перевести все проверки на справочники из датахаба, сейчас проверяется весь выгруженный фактический массив (коммент от 15.10.2025)
</div>

* партнеры

In [10]:
# меппинг партнеров
map_partner = bf.pl_open_excel(map_path, 'partner')
# уникальные партнеры
partner_unique = df_x_promo_all.\
   select(['lty_partner_nm']).\
   unique()
partner_owner_unique = df_x_promo_all.\
   select(['lty_partner_owner_nm']).\
   unique()
partner_total_unique = partner_owner_unique.\
   join(partner_unique, 
        left_on = 'lty_partner_owner_nm', 
        right_on = 'lty_partner_nm', 
        how = 'full', 
        coalesce = True).\
   filter(pl.col('lty_partner_owner_nm').is_not_null()).\
   rename({'lty_partner_owner_nm':'lty_partner_nm'})
# проверка, что у нас все присутствует
check_partner = partner_total_unique.join(map_partner, on = 'lty_partner_nm', how = 'anti')
bf.check_lf_if_null(check_partner)

Таблица пустая


* уровни

In [12]:
# меппинг уровней
map_chain = bf.pl_open_excel(map_path, 'levels')
# уникальные цепочек
chain_unique = df_x_promo_all.\
   select(['lty_promotion_action_chain_nm']).\
   unique().\
   filter(pl.col('lty_promotion_action_chain_nm').is_not_null())
# проверка, что у нас все присутствует
check_chain = chain_unique.join(map_chain, on = 'lty_promotion_action_chain_nm', how = 'anti')
bf.check_lf_if_null(check_chain)

Таблица пустая


* источники начислений

In [13]:
# меппинг источников начислений
map_promo_owner = bf.pl_open_excel(map_path, 'promo_owner')
# уникальные источники
promo_owner_unique = df_x_promo_all.\
   select(['lty_promo_owner_iss_owner_nm']).\
   unique().\
   filter(pl.col('lty_promo_owner_iss_owner_nm').is_not_null())
# проверка, что у нас все присутствует
check_promo_owner = promo_owner_unique.join(map_promo_owner, on = 'lty_promo_owner_iss_owner_nm', how = 'anti')
bf.check_lf_if_null(check_promo_owner)

Таблица пустая


* технические магазины

In [14]:
# меппинг технических магазинов
map_store_tech = bf.pl_open_excel(map_path, 'store_tech')
# уникальные тех магазины
store_tech_unique = df_x_promo_all.\
   filter(pl.col('lty_partner_nm').is_in(['Пятерочка','Перекресток'])).\
   select(['store_tech_mvnt_nm']).\
   unique()
store_tech_owner_unique = df_x_promo_all.\
   filter(pl.col('lty_partner_owner_nm').is_in(['Пятерочка','Перекресток','Сервис Геймификации'])).\
   select(['store_tech_owner_desc']).\
   unique()
store_tech_total_unique = store_tech_owner_unique.\
   join(store_tech_unique, 
        left_on = 'store_tech_owner_desc', 
        right_on = 'store_tech_mvnt_nm', 
        how = 'full', 
        coalesce = True).\
   filter(pl.col('store_tech_owner_desc').is_not_null()).\
   rename({'store_tech_owner_desc':'store_tech_desc'})
# проверка, что у нас все присутствует
check_store_tech = store_tech_total_unique.join(map_store_tech, on = 'store_tech_desc', how = 'anti')
bf.check_lf_if_null(check_store_tech)

Таблица пустая


* регионы

In [16]:
# меппинг регионов
map_reg = bf.pl_open_excel(map_path, 'region')
# уникальные регионы
reg_unique = df_x_promo_all.\
   select(['division_nm']).\
   unique().\
   filter(pl.col('division_nm').is_not_null())
# проверка, что у нас все присутствует
check_reg = reg_unique.join(map_reg, on = 'division_nm', how = 'anti')
bf.check_lf_if_null(check_reg)

Таблица пустая


#### Создаем итоговую таблицу с меппингами

In [17]:
# применяем все имеющиеся меппинги
df_x_promo = df_x_promo_all.\
   join(map_mvnt_type, on = 'lty_point_mvnt_type_desc', how = 'left', suffix = '_mvnt').\
   join(map_partner, left_on = 'lty_partner_owner_nm', right_on = 'lty_partner_nm', how = 'left', suffix = '_po').\
   join(map_partner, left_on = 'lty_partner_nm', right_on = 'lty_partner_nm', how = 'left', suffix = '_p').\
   join(map_store_tech, left_on = 'store_tech_owner_desc', right_on = 'store_tech_desc', how = 'left', suffix = '_sto').\
   join(map_store_tech, left_on = 'store_tech_mvnt_nm', right_on = 'store_tech_desc', how = 'left', suffix = '_st').\
   join(map_chain, on = 'lty_promotion_action_chain_nm', how = 'left', suffix = '_c').\
   join(map_promo_owner, on = 'lty_promo_owner_iss_owner_nm', how = 'left', suffix = '_prm').\
   join(map_reg, on = 'division_nm', how = 'left', suffix = '_reg').\
   join(map_manual, on = 'loyalty_operation_batch_id', how = 'left', suffix = '_mnl')
# создаем колясные столбцы
df_x_promo = df_x_promo.with_columns(
   lty_promo_owner_iss_owner_nm_cl = pl.when(pl.col('loyalty_operation_batch_id').is_not_null()).\
       then(pl.lit('Ручное начисление')).\
       otherwise(pl.coalesce(pl.col('lty_promo_owner_iss_owner_nm'),pl.lit(na_txt))),
   lty_promotion_iss_owner_nm_cl = pl.coalesce(pl.col('lty_promotion_iss_owner_nm'), pl.col('batch_comment')),
   lty_partner_owner_nm_cl = pl.coalesce(pl.col('lty_partner_nm_corr'), pl.col('lty_partner_owner_nm'), pl.lit(na_txt)),
   lty_partner_owner_nm_ext_cl = pl.coalesce(pl.col('store_tech_name'), pl.col('lty_partner_nm_corr'), pl.col('lty_partner_owner_nm'), pl.lit(na_txt)),
   lty_partner_nm_cl = pl.coalesce(pl.col('lty_partner_nm_corr_p'), pl.col('lty_partner_nm'), pl.lit(na_txt)),
   lty_partner_nm_ext_cl = pl.coalesce(pl.col('store_tech_name_st'), pl.col('lty_partner_nm_corr_p'), pl.col('lty_partner_nm'), pl.lit(na_txt)),
   promo_owner_group_cl = pl.coalesce(pl.col('partner_owner_group'), pl.col('promo_owner_group'), pl.col('promo_nm_group'), pl.lit(other_txt)),
   promo_group_budget_tcx_cl = pl.coalesce(pl.col('po_group_budget_tcx'), pl.col('pn_group_budget_tcx'), pl.lit(other_txt)),
   promo_group_budget_tc5_cl = pl.coalesce(pl.col('po_group_budget_tc5'), pl.col('pn_group_budget_tc5'), pl.lit(other_txt))
)

In [ ]:
# итоговые столбцы
total_table_cols = ['dt_month',
                   'lty_promo_owner_iss_owner_nm_cl',
                   'lty_promotion_iss_owner_nm_cl',
                   'lty_point_duration_desc',
                   'lty_point_mvnt_type_desc',
                   'mvnt_type_group_1',
                   'mvnt_type_group_2',
                   'lty_partner_owner_nm_cl',
                   'lty_partner_owner_nm_ext_cl',
                   'lty_partner_nm_cl',
                   'lty_partner_nm_ext_cl',
                   'macroregion_nm',
                   'division_nm',
                   'territory_nm',
                   'lty_promotion_action_chain_nm',
                   'level_n',
                   'partner_group',
                   'partner_group_p',
                   'promo_owner_group_cl',
                   'promo_group_budget_tcx_cl',
                   'promo_group_budget_tc5_cl']
# агрегация
df_x_promo = df_x_promo.group_by(total_table_cols).agg(lty_point_rub_sum = pl.col('lty_point_rub_amt').sum())
# переименование столбцов
df_x_promo = df_x_promo.rename({'lty_promo_owner_iss_owner_nm_cl':'lty_promo_owner_iss_owner_nm',
                               'lty_promotion_iss_owner_nm_cl':'lty_promotion_iss_owner_nm',
                               'lty_partner_owner_nm_cl':'lty_partner_owner_nm',
                               'lty_partner_owner_nm_ext_cl':'lty_partner_owner_nm_ext',
                               'lty_partner_nm_cl':'lty_partner_nm',
                               'lty_partner_nm_ext_cl':'lty_partner_nm_ext',
                               'promo_owner_group_cl':'promo_owner_group',
                               'promo_group_budget_tcx_cl':'promo_group_budget_tcx',
                               'promo_group_budget_tc5_cl':'promo_group_budget_tc5',
                               'partner_group':'partner_owner_group',
                               'partner_group_p':'partner_group'
                              }
                             )
# проверяем
df_x_promo.collect_schema()

#### Сохраняем в результатирующую таблицу CSV

In [19]:
for y in df_x_promo_all_years:
   print(f'Обработка файла с {y} годом')
   # таблица для сохранения
   output_data = df_x_promo.filter(pl.col('dt_month').dt.year() == y).collect()
   # сохранение csv сжатых файлов
   with gzip.open(path_root_x + r'\reporting\csv_files\df_x_promo_' + str(y) + '.csv.gz', 'wb') as f:
       output_data.write_csv(f, separator = ';')
        print(f'-- файл CSV сохранен')
   # сохранение parquet файлов
   output_data.write_parquet(path_root_x + r'\reporting\parquet_files\x_promo_' + str(y) + '.parquet')
   print(f'-- файл Parquet сохранен')
   # очищаем
   output_data = None
   gc.collect()

Обработка файла с 2023 годом
-- файл CSV сохранен
-- файл Parquet сохранен
Обработка файла с 2024 годом
-- файл CSV сохранен
-- файл Parquet сохранен
Обработка файла с 2025 годом
-- файл CSV сохранен
-- файл Parquet сохранен


#### Выгружаем из Parquet
Если файл уже был собран ранее, для отчетов ниже

In [ ]:
df_x_promo = pl.scan_parquet(r'P:\*\reports\10_sources\data\1_x_promo_w_geo\reporting\parquet_files')
df_x_promo.collect_schema()

## РТО

Если нужно забрать уже готовый отчет

In [23]:
# загружаем предобработанный (если требуется) файл
qlik_rto_report = pl.scan_csv(qlik_rto_report_path, separator = ';')
# проверяем набор данных
qlik_rto_report.collect_schema()

Если формируем новый

In [ ]:
# меппинг по РТО
map_rto_s = bf.pl_open_excel(map_path, 'rto_source')
# папка с файлами csv
qlik_rto_raw_path = r'P:\*\reports\10_sources\data\2_qlik_scmx01\raw\*xlsx'
qlik_rto_report_path = r'P:\*\reports\10_sources\data\2_qlik_scmx01\reporting'

In [ ]:
# соединяем все файлы
qlik_rto_raw = bf.read_all_excel_files(qlik_rto_raw_path)
# добавляем меппинг
qlik_rto_report = qlik_rto_raw.\
   join(map_rto_s, left_on = 'Онлайн-проект', right_on = 'rtl_txn_source_code', how = 'left').\
   join(map_partner.select(['lty_partner_nm', 'lty_partner_nm_corr']), left_on = 'Формат ЦФО', right_on = 'lty_partner_nm', how = 'left', suffix = '_prtnr').\
   select(pl.exclude(['prtnr_id','rto_id','reg_id']))

Сохранение результата

In [ ]:
# сохраняем результат в файл
bf.pl_save_csv(qlik_rto_report, qlik_rto_report_path, 'rto_qlik_src', cur_dt = False)

In [ ]:
# сохраняем отчет также для отчетности мониторинга бюджета
rto_monitoring_path_tcx = r'P:\*\!Бюджет ТСХ\Мониторинг бюджета\data_src'
bf.pl_save_excel(qlik_rto_report.filter(pl.col('lty_partner_nm_corr')=='Перекресток'), rto_monitoring_path_tcx, 'budg_rto', cur_dt = False)

## Отчетность

### Мониторинг бюджета ТСХ

In [18]:
# партнеры сети
tcx_partners = ['Перекресток','Обратная связь_TCX','Перекресток агрегаторы']
# фильтруем по партнерам владельцам
df_x_budg = df_x_promo.filter(pl.col('lty_partner_owner_nm').is_in(tcx_partners) | pl.col('lty_partner_nm').is_in(tcx_partners))
# добавление условных столбцов
df_x_budg = df_x_budg.with_columns(
   is_for_budget = pl.when(pl.col('lty_partner_owner_nm').\
       is_in(tcx_partners + ['Сервис Геймификации'])).\
       then(1).\
       otherwise(0)
)
# колонки группировки
df_x_budg_cols = ['dt_month',
                 'lty_promo_owner_iss_owner_nm',
                 'lty_promotion_iss_owner_nm',
                 'lty_point_duration_desc',
                 'lty_point_mvnt_type_desc',
                 'mvnt_type_group_1',
                 'mvnt_type_group_2',
                 'lty_partner_owner_nm',
                 'lty_partner_owner_nm_ext',
                 'lty_partner_nm',
                 'lty_partner_nm_ext',
                 'partner_group',
                 'partner_owner_group',
                 'lty_promotion_action_chain_nm',
                 'level_n',
                 'promo_owner_group',
                 'promo_group_budget_tcx',
                 'is_for_budget']
# агрегация до нужных столбцов
df_x_budg = df_x_budg.\
   filter(pl.col('dt_month').dt.year()>=2024).\
   group_by(df_x_budg_cols).\
   agg(lty_point_rub_sum = pl.col('lty_point_rub_sum').sum())

In [19]:
# меппинг источников начислений для бюджета тсх
map_items = bf.pl_open_excel(map_path, 'budg_tcx_25')
# уникальные источники
map_items_unique = df_x_budg.\
   select(['promo_group_budget_tcx']).\
   unique().\
   filter(pl.col('promo_group_budget_tcx').is_not_null())
# проверка, что у нас все присутствует
check_map_items = map_items_unique.join(map_items, left_on = 'promo_group_budget_tcx', right_on = 'item', how = 'anti')
check_map_items.collect()

promo_group_budget_tcx
str


In [20]:
# добавляем условие компенсации
df_x_budg = df_x_budg.\
   join(map_items, left_on = 'promo_group_budget_tcx', right_on = 'item', how = 'left').\
   select(pl.exclude(['bdg_tcx_id']))

In [21]:
# сохраняем результат
bf.pl_save_excel(df_x_budg, r'P:\*\!Бюджет ТСХ\Мониторинг бюджета\data_src', 'x_budg_loyalty', cur_dt = False)
# РТО сохраняем на предыдущем этапе обработки РТО

### Мониторинг бюджета ТС5

* файл движения баллов

In [26]:
# партнеры сети
tc5_partners = ['Пятерочка']
# фильтруем по партнерам владельцам
df_5_budg = df_x_promo.filter(pl.col('partner_owner_group').is_in(tc5_partners) | pl.col('partner_group').is_in(tc5_partners))
# добавление условных столбцов
df_5_budg = df_5_budg.with_columns(
   is_for_budget = pl.when(pl.col('partner_owner_group').\
       is_in(tc5_partners + ['Геймификация'])).\
       then(1).\
       otherwise(0)
)
# колонки группировки
df_5_budg_cols = ['dt_month',
                 'lty_promo_owner_iss_owner_nm',
                 'lty_promotion_iss_owner_nm',
                 'lty_point_duration_desc',
                 'lty_point_mvnt_type_desc',
                 'mvnt_type_group_1',
                 'mvnt_type_group_2',
                 'lty_partner_owner_nm',
                 'lty_partner_owner_nm_ext',
                 'lty_partner_nm',
                 'lty_partner_nm_ext',
                 'partner_group',
                 'partner_owner_group',
                 'lty_promotion_action_chain_nm',
                 'level_n',
                 'promo_owner_group',
                 'promo_group_budget_tc5',
                 'is_for_budget']
# агрегация до нужных столбцов
df_5_budg = df_5_budg.\
   filter(pl.col('dt_month').dt.year()>=2024).\
   group_by(df_5_budg_cols).\
   agg(lty_point_rub_sum = pl.col('lty_point_rub_sum').sum())

In [27]:
# сохраняем результат
bf.pl_save_excel(df_5_budg, r'\\*\FolderRedirection-6\*\Desktop\x5_work\projects_files\3_budget-2025\Слайд - балльная нагрузка\2025_09', 'x_5_budg_loyalty', cur_dt = False)

In [ ]:
# местоположение исходных данных
csv_path = r"\\*\FolderRedirection-6\*\Desktop\x5_work\projects_files\10_sources\raw_data\MOVEMENT_FIN"
all_files = glob.glob(os.path.join(csv_path, "*.csv"))

Помесячно

In [ ]:
%%time
# переменные
data_name = "closing_main_wo_sap"
raw_path_folder = raw_folder + "\\" + data_name # папка сохранения
proc_path_folder = path_folder + r"\processed_data" + "\\" + data_name # папка сохранения готовых данных

In [ ]:
# запуск запроса
bf.execute_range_queries(connection_str, datetime(2024,1,1).date(),datetime(2025,1,1).date(), path_sql, proj_n + data_name, raw_path_folder)

### Отчет по диджитал

In [13]:
# формируем данные по диджитал
df_digital = df_x_promo.\
   filter(pl.col('lty_partner_owner_nm_ext').is_in(['Пятерочка Доставка','Перекресток Доставка']) |
          pl.col('lty_partner_nm_ext').is_in(['Пятерочка Доставка','Перекресток Доставка'])
         )
# добавляем единый столбец по территории, для перека дивизион, пятерки территория
df_digital = df_digital.with_columns(
   region = pl.when(pl.col('macroregion_nm')=='Перекресток').then(pl.col('division_nm')).otherwise(pl.col('territory_nm'))
)

* агрегация

In [16]:
# столбцы группировки
df_digital_agg_cols = [
   'dt_month',
   'region',
   'lty_promo_owner_iss_owner_nm',
   'lty_promotion_iss_owner_nm',
   'lty_point_mvnt_type_desc',
   'lty_point_duration_desc',
   'promo_owner_group'
]
# фильтр ошибочных акций из начислений
df_digital_exclude = ['TCХ Открытие 2000 (2053,2019,2071)']
# дата начальная
date_start = datetime(2024,9,1)
# начисления
df_digital_in = df_digital.\
   filter(pl.col('dt_month')>=date_start).\
   filter(pl.col('lty_partner_owner_nm_ext').is_in(['Пятерочка Доставка','Перекресток Доставка'])).\
   filter(pl.col('lty_point_mvnt_type_desc').is_in(['Начисление'])).\
   filter(~pl.col('lty_promotion_iss_owner_nm').is_in(df_digital_exclude)).\
   group_by(df_digital_agg_cols+['lty_partner_owner_nm_ext']).\
   agg(lty_point_rub_sum = pl.col('lty_point_rub_sum').sum())
# списания
df_digital_out = df_digital.\
   filter(pl.col('dt_month')>=date_start).\
   filter(pl.col('lty_partner_nm_ext').is_in(['Пятерочка Доставка','Перекресток Доставка'])).\
   filter(pl.col('lty_point_mvnt_type_desc').is_in(['Списание'])).\
   group_by(df_digital_agg_cols+['lty_partner_nm_ext']).\
   agg(lty_point_rub_sum = pl.col('lty_point_rub_sum').sum())

In [ ]:
# папка для сохранения результата
digital_report_path = r'\\*\FolderRedirection-6\*\Desktop\x5_work\projects_files\1_digital\reporting'
# сохраняем результат в файл
bf.pl_save_csv(df_digital_in, digital_report_path, 'digital_in', cur_dt = False)
bf.pl_save_csv(df_digital_out, digital_report_path, 'digital_out', cur_dt = False)

## Тестирование Trino

In [55]:
# импортируем библиотеки
from trino.dbapi import connect as tr_conn
from trino.auth import BasicAuthentication
import urllib3
# отключаем уведомления
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
# создаем соединение
conn = tr_conn(
   host="*.x5.ru",
   port=443,
   auth=BasicAuthentication(f"{getuser().lower()}", f"{getpass(prompt='Пароль: ')}"),
   http_scheme="https",
   verify=False
)
# функция обращения к базе запросом и возврат результат в df
def get_data_from_cdh(qr):
   with conn.cursor() as cur:
       cur.execute(qr)
       rows = cur.fetchall()
       column_names = [desc[0] for desc in cur.description]
       trino_df = pd.DataFrame(rows, columns=column_names)
   return trino_df

Пароль: ········


In [ ]:
# запрос
get_data_from_cdh('SELECT * FROM cdh_ice* limit 1')